In [1]:
import numpy as np

In [2]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly import tools as tls

In [3]:
from scipy.spatial import Delaunay

In [4]:
def sq_norm(v): #squared norm 
    return np.linalg.norm(v)**2

Compute the circumcenter and circumradius of a triangle (see their definitions [here](https://en.wikipedia.org/wiki/Circumscribed_circle#Circumcircle_equations)):

In [5]:
def circumcircle(points,simplex):
    A=[points[simplex[k]] for k in range(3)]
    M=[[1.0]*4]
    M+=[[sq_norm(A[k]), A[k][0], A[k][1], 1.0 ] for k in range(3)] 
    M=np.asarray(M, dtype=np.float32) 
    S=np.array([0.5*np.linalg.det(M[1:,[0,2,3]]), -0.5*np.linalg.det(M[1:,[0,1,3]])]) 
    a=np.linalg.det(M[1:, 1:]) 
    b=np.linalg.det(M[1:, [0,1,2]]) 
    return S/a,  np.sqrt(b/a+sq_norm(S)/a**2) #center=S/a, radius=np.sqrt(b/a+sq_norm(S)/a**2)

Filter the Delaunay triangulation to get the $\alpha$-complex:

In [6]:
def get_alpha_complex(alpha, points, simplexes):
    #alpha is the parameter for the alpha shape
    #points are given data points 
    #simplexes is the  list of indices in the array of points 
    #that define 2-simplexes in the Delaunay triangulation
    
    return filter(lambda simplex: circumcircle(points,simplex)[1]<alpha, simplexes)

Get data for Plotly plot of a subcomplex of the Delaunay triangulation:

In [7]:
def Plotly_data(points, complex_s):
    #points are the given data points, 
    #complex_s is the list of indices in the array of points defining 2-simplexes(triangles) 
    #in the simplicial complex to be plotted
    X=[]
    Y=[]
    for s in complex_s:
        X+=[points[s[k]][0] for k in [0,1,2,0]]+[None]
        Y+=[points[s[k]][1] for k in [0,1,2,0]]+[None]
    return X,Y    

In [12]:
colors=['#C0223B', '#404ca0', 'rgba(173,216,230, 0.5)']# colors for vertices, edges and 2-simplexes

In [13]:
def make_trace(x, y,  point_color=colors[0], line_color=colors[1]):# define the trace
                                                                   #for an alpha complex
    return Scatter(mode='markers+lines', #set vertices and 
                                         #edges of the alpha-complex
                   name='',
                   x=x,
                   y=y,
                   marker=Marker(size=6.5, color=point_color),
                   line=Line(width=1.25, color=line_color),
                  )

In [14]:
def make_XAxis(axis_style):
    return XAxis(axis_style)
    
def make_YAxis(axis_style):
    return YAxis(axis_style)  

In [15]:
pts=np.loadtxt('data-ex-2d.txt')
tri = Delaunay(pts)

Subplots (https://plot.ly/python/subplots/):

In [16]:
figure = tls.make_subplots(rows=3, cols=2,
                           subplot_titles=('Delaunay triangulation', 'Alpha shape, alpha=0.10', 'Alpha shape, alpha=0.12', 'Alpha shape, alpha=0.14', 'Alpha shape, alpha=0.16', 'Alpha shape, alpha=0.18'),
                           horizontal_spacing=0.1, 
                          )

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



In [17]:
pl_width=800
pl_height=1000
title = 'Delaunay triangulation and Alpha Complex/Shape for a Set of 2D Points'

figure['layout'].update(title=title,                                 
                        font= Font(family="Open Sans, sans-serif"),
                        showlegend=False,     
                        hovermode='closest',  
                        autosize=False,       
                        width=pl_width,       
                        height=pl_height,
                        margin=Margin(
                            l=65,
                            r=65,
                            b=85,
                            t=120
                            ),           
                       shapes=[]
                       )

Set axis style:

In [18]:
axis_style = dict(showline=True, 
                  mirror=True,
                  zeroline=False,
                  showgrid=False,
                  showticklabels=True,
                  range=[-0.1,1.1],
                  tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1.0],
                  ticklen=5
                 )

In [19]:
for s in range(1,7): # from 1 to 'num of figures + 1'
    figure['layout'].update({'xaxis{}'.format(s): make_XAxis(axis_style)})# set xaxis style
    figure['layout'].update({'yaxis{}'.format(s): make_YAxis(axis_style)})# set yaxis style

Calcurate $\alpha$-complexes:

In [20]:
alpha_complex1=get_alpha_complex(0.10, pts, tri.simplices)
alpha_complex2=get_alpha_complex(0.12, pts, tri.simplices)
alpha_complex3=get_alpha_complex(0.14, pts, tri.simplices)
alpha_complex4=get_alpha_complex(0.16, pts, tri.simplices)
alpha_complex5=get_alpha_complex(0.18, pts, tri.simplices)

In [29]:
dir(alpha_complex1)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [21]:
X,Y=Plotly_data(pts, tri.simplices)# get data for Delaunay triangulation
figure.append_trace(make_trace(X, Y), 1, 1) 

X,Y=Plotly_data(pts, alpha_complex1)# data for alpha complex
figure.append_trace(make_trace(X, Y), 1, 2) 

X,Y=Plotly_data(pts, alpha_complex2)# data for alpha complex
figure.append_trace(make_trace(X, Y), 2, 1) 

X,Y=Plotly_data(pts, alpha_complex3)# data for alpha complex
figure.append_trace(make_trace(X, Y), 2, 2) 

X,Y=Plotly_data(pts, alpha_complex4)# data for alpha complex
figure.append_trace(make_trace(X, Y), 3, 1) 

X,Y=Plotly_data(pts, alpha_complex5)# data for alpha complex
figure.append_trace(make_trace(X, Y), 3, 2) 

In [26]:
for s in alpha_complex1: #fill in the triangles of the alpha complex
    A=pts[s[0]]
    B=pts[s[1]]
    C=pts[s[2]]
    figure['layout']['shapes'].append(dict(path='M '+str(A[0])+',' +str(A[1])+' '+'L '+\
                                                 str(B[0])+', '+str(B[1])+ ' '+'L '+\
                                                 str(C[0])+', '+str(C[1])+' Z',
                                           fillcolor=colors[2],
                                           line=Line(color=colors[1], width=1.25),
                                           xref='x2',
                                           yref='y2'
                                           )
                                     )

In [27]:
py.iplot(figure, filename='2D-AlphaS-ex', width=850)

In [ ]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()